In [23]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#beautifulsoup
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

import re

import json

import pandas as pd

In [24]:
driver = webdriver.Chrome(
)
driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx")

#等待頁面載入
try:
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, 'input#jud_jmain')
    ))
    input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
    input_case_reason.send_keys("殺人")
    input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
    input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")
    search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
    search_button.click()
    sleep(1)
except TimeoutException:
    print("can not search")

In [25]:
#等待iframe與下一頁按鈕
try:
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, "iframe#iframe-data")
    ))

    iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
    driver.switch_to.frame(iframe)
    page_url = driver.find_element(By.CSS_SELECTOR, 'a#hlNext').get_attribute('href')
    print(page_url)

except TimeoutException:
    print("not found...next_page_button")

https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in


In [26]:
#測試一下標籤
driver.get(page_url)
cssSelector = 'a#hlTitle'
case_url = driver.find_element(By.CSS_SELECTOR, cssSelector)
case_url.click()
# 如果這邊click之後，沒有等JSt生成，就直接page_source抓動態html的話，會抓不到，用標籤判斷根本沒用，標籤會在，只是沒有內容，看有沒有其他隱性等待的方法(這樣比較保險)，不然就要強制ˋ等待了，或使隱性等待?(但他是如何判斷加載完成的...)
#!!!!要抓li，js生成的是li，因為平常進網頁就載入好了，跟一班明顯的js變化不同，所以沒有察覺
try:
    # 看有沒有抓到js生成的側攔
    sidebarSelector = 'ul.rela-law li'
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, sidebarSelector)
    ))

    #有的話，就取得動態html
    case_html = driver.page_source
    soup = bs(case_html, "lxml")

except TimeoutException:
    print("not found...element")


# side_bar = soup.select_one('div.rela-area.col-xs-4')
# print(side_bar.get_text())
### 這邊要改li然後再加進一個Lsit當中
linesSelector = 'ul.rela-law li'
lines = soup.select(linesSelector)
for law in lines:
    print(law.get_text())



家庭暴力防治法 第 2、63.1 條（104.02.04）
刑事訴訟法 第 159、159.5、299 條（106.11.16）
中華民國刑法 第 1、25、38、51、57、271 條（105.11.30）


In [27]:
# 若case_id為"本件無歷審裁判"，則不會有url
history_judgement = soup.select("div#JudHis div.panel-body ul li")
history_judgement_list = []
for index in range(0, len(history_judgement)):
    if not str(history_judgement[index].select_one('a')) == "None":
        history_judgement_list.append({
        "case_id":str(history_judgement[index].get_text()),
        "url":"https://judgment.judicial.gov.tw/FJUD/"+ str(history_judgement[index].select_one('a')['href'])
        #抓不到a就會變成NoneType object，要怎麼處理??
        # NoneType object當然也不能用['href']所以會報錯

        # "url":"https://judgment.judicial.gov.tw/FJUD/"+ history_judgement[index].select_one('a')['href']
        })
    else:
        history_judgement_list.append({
        "case_id":str(history_judgement[index].get_text())
        })

history_judgement_list

[{'case_id': '本件無歷審裁判'}]

In [28]:
page_url_list = [] 
regex_page_url = "&page=[\d]*"
test = re.split(regex_page_url, page_url)
for each_page in range(1,26):
    page_url_list.append(f"{test[0]}&page={each_page}{test[1]}") 

In [29]:
# 放所有case的資料
all_case_list = []

In [30]:
# 加try excecpt
# 要設一些如果沒抓到東西時的條件，比如歷史審判可能會沒有
# 然後改移下一開始的wait反正就是等旁偏那個javascript生出來的測攔出來就好
# 開始爬case，這是每一頁的網址
for page in page_url_list:
    driver.get(page)
    try:
        cssSelector = 'a#hlTitle'
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, cssSelector)
        ))

        case_url = driver.find_elements(By.CSS_SELECTOR, cssSelector)
    except TimeoutException:
        print("cases not found")
        break

    # cases找到後開始一個一個點
    # 這是每一頁當中的至多20筆的資料    
    for case in case_url:
        case.click()

        try:
            # 看有沒有抓到js生成的側攔"當中的li"，li就是js後來生成的
            # 這邊要改可能還是沒有laws，而且結果超奇怪....根本抓不到law結果欄位有值，抓地到history但沒有值...
            
            #還是這邊可以等兩個?等個兩個載好在來取得動態html
            side_li_selector_1 = 'ul.rela-law li'
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
            (By.CSS_SELECTOR, side_li_selector_1)
            ))

            side_li_selector_2 = 'div#JudHis div.panel-body ul li'
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
            (By.CSS_SELECTOR, side_li_selector_2)
            ))

            #有的話，就取得動態html
            case_html = driver.page_source
            soup = bs(case_html, "lxml")

        except TimeoutException:
            print("not found...side_bar, data shortage")
            continue #這是用在迴圈的，也就是說如果沒有應該會直接跑下一個case才對，但也些按件，有可能就少一項也不影響阿...但我下面是寫在外面的無論如何都會繼續，所以law_list也寫一個判斷條件好了，這邊要想一下，沒抓到這些東西，也可以爬一些資料只是會少而已，還是就一樣抱錯，然後取得靜態case_html

        allSelector = 'div.col-td'
        all = soup.select(allSelector)

        laws_list = []
        linesSelector = 'ul.rela-law'
        lines = soup.select(linesSelector)
        for law in lines:
            laws_list.append(str(law.get_text()))

        
        history_judgement = soup.select("div#JudHis div.panel-body ul li")
        history_judgement_list = []
        for index in range(0, len(history_judgement)):
            if not str(history_judgement[index].select_one('a')) == "None":
                history_judgement_list.append({
                    "case_id":str(history_judgement[index].get_text()),
                    "url":"https://judgment.judicial.gov.tw/FJUD/"+ str(history_judgement[index].select_one('a')['href'])
                })
            else:
                history_judgement_list.append({
                "case_id":str(history_judgement[index].get_text())
                })


        all_case_list.append({
        "case_id":str(all[0].get_text().strip()),
        "date":str(all[1].get_text().strip()),
        "simple_reason":str(all[2].get_text().strip()),
        "laws":laws_list,
        "history_judgement":history_judgement_list,
        "context":str(all[3].get_text()),
        })

        driver.back()


In [31]:
print(all_case_list[44]['laws'])

['刑事訴訟法 第 159.5、299 條（106.11.16）中華民國刑法 第 1、3、19、25、38、87、271 條（105.11.30）']


In [32]:
print(len(all_case_list))

500


In [33]:
# 就寫成dataframe再轉成csv八
tt = pd.DataFrame(all_case_list)
tt.to_csv('./cases_500_v2.csv')

In [36]:
tt

,case_id,date,simple_reason,laws,history_judgement,context
0,臺灣高等法院 臺中分院 113 年度上訴字第 213 號刑事判決,民國 113 年 04 月 24 日,殺人未遂等,[刑事訴訟法 第 159.1、369 條（106.11.16）中華民國刑法 第 1、13、2...,[{'case_id': '臺灣臺中地方法院 112 年度 訴 字第 1877 號裁定(11...,\n\n\n\n 臺灣高等法院臺中分院刑事判決113年度上訴字第213號上 訴 人 臺...
1,臺灣高等法院 花蓮分院 113 年度上訴字第 17 號刑事判決,民國 113 年 04 月 19 日,殺人未遂,[刑事訴訟法 第 348、369 條（106.11.16）中華民國刑法 第 1、19、25、...,[{'case_id': '臺灣花蓮地方法院 112 年度 訴 字第 113 號裁定(112...,\n\n\n\n臺灣高等法院花蓮分院刑事判決113年度上訴字第17號上 訴 人 即 ...
2,臺灣高等法院 112 年度上更一字第 104 號刑事判決,民國 113 年 04 月 18 日,殺人,[刑事訴訟法 第 159、159.4、159.5、348、369 條（106.11.16）中...,[{'case_id': '臺灣新北地方法院 111 年度 重訴 字第 4 號裁定(111....,\n\n\n\n臺灣高等法院刑事判決112年度上更一字第104號上 訴 人 臺灣新北地...
3,臺灣臺南地方法院 113 年度訴字第 150 號刑事判決,民國 113 年 04 月 17 日,家暴殺人未遂等,[家庭暴力防治法 第 2、3 條（104.02.04）兒童及少年福利與權益保障法 第 69、...,[{'case_id': '臺灣臺南地方法院 113 年度 訴 字第 150 號裁定(113...,\n\n\n\n臺灣臺南地方法院刑事判決113年度訴字第150號公 訴 人 臺灣臺南地...
4,臺灣高等法院 112 年度上更一字第 123 號刑事判決,民國 113 年 04 月 11 日,殺人未遂等,[刑事訴訟法 第 158.4、159.5、369、371 條（106.11.16）中華民國刑...,[{'case_id': '臺灣桃園地方法院 111 年度 原訴 字第 91 號判決(111...,\n\n\n\n臺灣高等法院刑事判決112年度上更一字第123號上 訴 人 臺灣桃園地...
...,...,...,...,...,...,...
495,臺灣高等法院 臺中分院 109 年度上訴字第 2236 號刑事判決,民國 111 年 03 月 16 日,槍砲彈藥刀械管制條例案等,[槍砲彈藥刀械管制條例 第 7、8、12、18 條（106.06.14）刑事訴訟法 第 15...,[{'case_id': '臺灣臺中地方法院 109 年度 訴 字第 745 號判決(109...,\n\n\n\n臺灣高等法院臺中分院刑事判決109年度上訴字第2236號上 訴 人 臺...
496,臺灣高等法院 臺南分院 110 年度上訴字第 1144 號刑事判決,民國 111 年 03 月 16 日,殺人未遂,[刑事訴訟法 第 158.4、159、159.1、159.5、369 條（106.11.16...,[{'case_id': '臺灣嘉義地方法院 109 年度 訴 字第 353 號判決(110...,\n\n\n\n臺灣高等法院臺南分院刑事判決110年度上訴字第1144號上 訴 人 臺...
497,臺灣臺北地方法院 110 年度重訴字第 15 號刑事判決,民國 111 年 03 月 11 日,殺人,[家庭暴力防治法 第 2、3 條（104.02.04）刑事訴訟法 第 159、159.1、1...,[{'case_id': '臺灣臺北地方法院 110 年度 重訴 字第 15 號判決(111...,\n\n\n\n 臺灣臺北地方法院刑事判決 110...
498,臺灣高等法院 110 年度上訴字第 2664 號刑事判決,民國 111 年 03 月 09 日,殺人,[刑事訴訟法 第 159.1、159.5、299、348、369 條（106.11.16）中...,[{'case_id': '臺灣新北地方法院 110 年度 重訴 字第 4 號裁定(110....,\n\n\n\n臺灣高等法院刑事判決110年度上訴字第2664號上 訴 人 臺灣新北地...


In [ ]:
# side_bar兩個都要判斷?
# div#JudHis > div.panel-body > ul > li